<a href="https://colab.research.google.com/github/jlayne246/Resume-Analyser/blob/ner/Resume_Fine_TuningModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install -q transformers datasets seqeval accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "yashpwr/resume-ner-bert-v2"  # or "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

Creating Synthetic Data

In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.7 MB/s eta 0:00:00


In [ ]:
"""
from faker import Faker
import random
import json

fake = Faker('en_US')  # Western locale

# Define some labels
ENTITY_LABELS = ["NAME", "COMPANIES_WORKED_AT", "LOCATION", "DEGREE", "COLLEGE_NAME", "SKILLS"]

# Example skills pool
SKILLS_POOL = [
    "Python", "JavaScript", "SQL", "C++", "AWS", "Docker", "Kubernetes",
    "Machine Learning", "Data Analysis", "Project Management"
]

def generate_resume():
    # Generate basic fields
    name = fake.name()
    company = fake.company()
    city = fake.city()
    state = fake.state()
    degree = random.choice(["Bachelor of Science", "Master of Science", "BBA", "MBA"])
    college = fake.company() + " University"
    skills = random.sample(SKILLS_POOL, k=random.randint(3, 6))

    # Employment period
    year_start = random.randint(2015, 2020)
    year_end = year_start + random.randint(1, 5)

    # Construct resume text
    content = (
        f"{name} worked at {company} in {city}, {state} from {year_start} to {year_end}. "
        f"{name} graduated with a {degree} from {college}. "
        f"Skills include {', '.join(skills)}."
    )

    # Build annotation spans
    annotations = []

    def find_span(text, substring, label):
        start = text.find(substring)
        if start == -1:
            return None
        end = start + len(substring)
        return {"label": [label], "points": [{"start": start, "end": end}]}

    # Add annotations
    for substring, label in [
        (name, "NAME"),
        (company, "COMPANIES_WORKED_AT"),
        (f"{city}, {state}", "LOCATION"),
        (degree, "DEGREE"),
        (college, "COLLEGE_NAME"),
        (", ".join(skills), "SKILLS")
    ]:
        span = find_span(content, substring, label)
        if span:
            annotations.append(span)

    return {"content": content, "annotation": annotations}

# Generate multiple resumes
NUM_RESUMES = 500
synthetic_data = [generate_resume() for _ in range(NUM_RESUMES)]

# Save to JSONL for use in your pipeline
with open("synthetic_data.json", "w", encoding="utf-8") as f:
    for resume in synthetic_data:
        f.write(json.dumps(resume) + "\n")

print(f"✅ Generated {NUM_RESUMES} synthetic resumes.")

"""

from faker import Faker
import random
import json

fake = Faker()
Faker.seed(42)

entity_labels = [
    "Name", "Email Address", "Mobile Number",
    "Companies worked at", "College Name",
    "Degree", "Designation", "Skills", "Location"
]

synthetic_data = []

def add_entity(entities, text, label, value):
    """Helper to safely add entity spans by text lookup."""
    start = text.find(value)
    if start != -1:
        entities.append({
            "label": [label],
            "points": [{"start": start, "end": start + len(value)}]
        })

for _ in range(500):
    name = fake.name()
    email = fake.email()
    mobile = fake.msisdn()[:10]
    college = fake.company() + " College of Engineering"
    degree = random.choice([
        "B.E. in Computer Engineering",
        "B.Tech in Information Technology",
        "B.Tech in Computer Science",
        "B.Sc in Computer Science",
        "M.Tech in Data Science"
    ])
    company = fake.company()
    designations = random.sample([
        "Manager", "Technical Content Writer", "Data Engineer",
        "Software Developer", "System Administrator",
        "Machine Learning Intern", "Research Assistant"
    ], k=random.randint(1, 2))
    skills = random.sample([
        "Python", "Java", "C++", "Linux", "GitHub", "Django", "Docker",
        "AWS", "Excel", "PowerBI", "Automation", "TensorFlow", "Pandas"
    ], k=random.randint(5, 10))
    location = fake.city() + ", " + fake.state()

    # Generate "resume" with formatting
    text = f"""
{name}
Email: {email} | Mobile: {mobile}

Education
----------
{degree}
{college}

Experience
----------
{company}
{' / '.join(designations)}

Skills
------
- {', '.join(skills)}

Summary
-------
Enthusiastic professional with experience in {random.choice(skills)} and {random.choice(skills)}.
    """.strip()

    # Collect entity spans
    entities = []
    add_entity(entities, text, "Name", name)
    add_entity(entities, text, "Email Address", email)
    add_entity(entities, text, "Mobile Number", mobile)
    add_entity(entities, text, "College Name", college)
    add_entity(entities, text, "Degree", degree)
    add_entity(entities, text, "Companies worked at", company)
    for d in designations:
        add_entity(entities, text, "Designation", d)
    for s in skills:
        add_entity(entities, text, "Skills", s)
    add_entity(entities, text, "Location", location)

    synthetic_data.append({
        "content": text,
        "annotation": entities  # ✅ consistent with your converter
    })

# Save as line-delimited JSON
with open("synthetic_resume_ner.json", "w", encoding="utf-8") as f:
    for item in synthetic_data:
        f.write(json.dumps(item) + "\n")

print(f"✅ Generated {len(synthetic_data)} structured resumes with realistic layout.")



✅ Generated 500 structured resumes with realistic layout.


In [ ]:
print(synthetic_data[0])

{'content': 'Allison Hill\nEmail: donaldgarcia@example.net | Mobile: 9600133890\n\nEducation\n----------\nB.Tech IN INFORMATION TECHNOLOGY\nRobinson-Lawrence College of Engineering\n\nExperience\n----------\nBlake and Sons\nTechnical Content Writer\n\nSkills\n------\n- Django, PowerBI, Automation, Java, AWS, TensorFlow, Python, C++, Docker, GitHub\n\nSummary\n-------\nEnthusiastic professional with experience in GitHub and Java.', 'annotation': [{'label': ['Name'], 'points': [{'start': 0, 'end': 12}]}, {'label': ['Email Address'], 'points': [{'start': 20, 'end': 44}]}, {'label': ['Mobile Number'], 'points': [{'start': 55, 'end': 65}]}, {'label': ['College Name'], 'points': [{'start': 121, 'end': 161}]}, {'label': ['Degree'], 'points': [{'start': 88, 'end': 120}]}, {'label': ['Companies worked at'], 'points': [{'start': 185, 'end': 199}]}, {'label': ['Designation'], 'points': [{'start': 200, 'end': 224}]}, {'label': ['Skills'], 'points': [{'start': 242, 'end': 248}]}, {'label': ['Skills

Loading Dataset

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()  # choose kaggle.json


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"joshualayne","key":"feaced9af663f7b6162b758c425d5e8e"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d dataturks/resume-entities-for-ner
!unzip -q resume-entities-for-ner.zip

Dataset URL: https://www.kaggle.com/datasets/dataturks/resume-entities-for-ner
License(s): unknown
  0% 0.00/323k [00:00<?, ?B/s]
100% 323k/323k [00:00<00:00, 577MB/s]


In [ ]:
from datasets import load_dataset

kaggle_data = load_dataset("json", data_files="/content/Entity Recognition in Resumes.json")
kaggle_data = kaggle_data["train"].train_test_split(test_size=0.2)


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(kaggle_data["train"][0])


{'content': "Anurag Asthana\nPune, Maharashtra - Email me on Indeed: indeed.com/r/Anurag-Asthana/ea7451b2bdb6115a\n\n• Looking forward for joining a company where my creative thinking and excellent skills in\napplication development using modern development tools will be utilized.\n\nWilling to relocate: Anywhere\n\nWORK EXPERIENCE\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nApril 2017 to Present\n\nLocation: Pune, India\nClient: Insurance Client from London\nDuration: April 2017 -Till Date\nTechnology/Tools: Microsoft SharePoint 2013, Azure Cognitive Services, Azure Service Bus, Azure\nBlob Storage, .net framework 4.7\n\nResponsibilities:\n\n• Independently worked on designing solution architecture of the project.\n• Lead team from technical front in all the components built\n• Followed Agile development approach to efficiently manage continuous development process\nand incremental requirement changes\n• Design SharePoint components using SharePoint CSOM and .net framework 4.7\n• Design Az

In [ ]:
import json
from transformers import AutoTokenizer
from tqdm import tqdm

# Load your Kaggle dataset
data = []
with open("/content/Entity Recognition in Resumes.json", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

print(data[0])

# ✅ Combine with your synthetic data (which uses "text" + "entities" keys)
# So we unify the structure for consistency
def convert_synthetic_to_kaggle_format(synthetic_data):
    converted = []
    for item in synthetic_data:
        ann = []
        # The key should be 'annotation' based on the synthetic data generation
        for e in item["annotation"]:
            # The structure of synthetic data's annotation is different, adapt extraction
            if e.get("label") and e.get("points"):
                #label = e["label"][0].replace(" ", "_").upper()
                label = e["label"][0]
                for point in e["points"]:
                     # Handle missing start/end safely
                    start = point.get("start")
                    end = point.get("end")
                    if start is not None and end is not None:
                        ann.append({
                            "label": [label],
                            "points": [{"start": start, "end": end}]
                        })

        # Synthetic data structure has 'content' and 'annotation' directly
        converted.append({
            "content": item["content"],
            "annotation": ann
        })
    return converted

# Assume synthetic_data is available from a previous cell's execution
# If not, you might need to load it again
# with open("synthetic_data.json", "r", encoding="utf-8") as f:
#     synthetic_data = [json.loads(line) for line in f]


synthetic_converted = convert_synthetic_to_kaggle_format(synthetic_data)
print(synthetic_converted[0])

data = data + synthetic_converted

unique_labels = set()
for example in data:
    for ann in example["annotation"]:
        unique_labels.update(ann["label"])
print(sorted(unique_labels))

print(f"✅ Combined dataset size: {len(data)} examples")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def char_to_token_labels(example):
    text = example["content"]
    annotations = example.get("annotation", [])

    entities = []
    for ann in annotations:
        if ann.get("label") and ann.get("points"):  # Ensure label list isn’t empty and points exist
            label = ann["label"][0].replace(" ", "_").upper()
            for point in ann["points"]:
                # ✅ Handle missing start/end safely
                start = point.get("start")
                end = point.get("end")
                if start is not None and end is not None:
                    entities.append({
                        "start": start,
                        "end": end,
                        "label": label
                    })

    entities = sorted(entities, key=lambda x: x["start"])

    # Tokenize with offsets
    encoding = tokenizer(text, return_offsets_mapping=True, truncation=True, max_length=512)
    offsets = encoding["offset_mapping"]
    labels = ["O"] * len(offsets)

    for ent in entities:
        for i, (start, end) in enumerate(offsets):
            if start >= ent["start"] and end <= ent["end"]:
                if start == ent["start"]:
                    labels[i] = "B-" + ent["label"]
                else:
                    labels[i] = "I-" + ent["label"]

    tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])
    return {"tokens": tokens, "ner_tags": labels}

# Process all examples
processed = [char_to_token_labels(d) for d in tqdm(data)]

# Save processed dataset
with open("resume_ner_processed.json", "w", encoding="utf-8") as f:
    json.dump(processed, f, indent=2)

print("✅ Saved processed dataset to resume_ner_processed.json")

{'content': "Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills\nand knowledge for my individual and company's growth in best possible ways.\n\nWilling to relocate to: Bangalore, Karnataka\n\nWORK EXPERIENCE\n\nApplication Development Associate\n\nAccenture -\n\nNovember 2017 to Present\n\nRole: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries\nfor the Bot which will be triggered based on given input. Also, Training the bot for different possible\nutterances (Both positive and negative), which will be given as\ninput by the user.\n\nEDUCATION\n\nB.E in Information science and engineering\n\nB.v.b college of engineering and technology -  Hubli, Karnataka\n\nAugust 2013 to June 2017\n\n12th in Mathematics\n\nWoodbine modern school\n\nApril 2011 to March 2013\n\n10th\n\nKendriya 

100%|██████████| 720/720 [00:02<00:00, 348.95it/s]


✅ Saved processed dataset to resume_ner_processed.json


In [ ]:
import json
from tqdm import tqdm
from transformers import AutoTokenizer
from datasets import Dataset, ClassLabel, Features, Sequence, Value

# Load your processed dataset (already includes synthetic + real data)
with open("/content/resume_ner_processed.json", "r", encoding="utf-8") as f:
    processed_data = json.load(f)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Collect all unique labels from the processed data
all_labels = set()
for ex in processed_data:
    for tag in ex["ner_tags"]:
        all_labels.add(tag)
all_labels = sorted(list(all_labels))

label2id = {label: i for i, label in enumerate(all_labels)}
id2label = {i: label for label, i in label2id.items()}

def encode_example(example):
    # Convert tokens to IDs
    encoding = tokenizer(
        example["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # Align NER tag IDs with token positions
    labels = [label2id[tag] for tag in example["ner_tags"]]
    if len(labels) < len(encoding["input_ids"]):
        labels += [label2id["O"]] * (len(encoding["input_ids"]) - len(labels))
    else:
        labels = labels[:len(encoding["input_ids"])]

    encoding["labels"] = labels
    return encoding

# Tokenize the entire dataset
print("🔄 Tokenizing dataset...")
tokenized_data = [encode_example(ex) for ex in tqdm(processed_data)]

# Define dataset features
features = Features({
    "input_ids": Sequence(feature=Value("int64")),
    "token_type_ids": Sequence(feature=Value("int64")),
    "attention_mask": Sequence(feature=Value("int64")),
    "labels": Sequence(feature=ClassLabel(names=all_labels))
})

# Create Hugging Face Dataset
dataset = Dataset.from_list(tokenized_data)
dataset = dataset.cast(features)

# Save for reuse
dataset.save_to_disk("resume_ner_dataset")
print("✅ Dataset saved to 'resume_ner_dataset'")


🔄 Tokenizing dataset...


100%|██████████| 720/720 [00:01<00:00, 371.31it/s]


Casting the dataset:   0%|          | 0/720 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/720 [00:00<?, ? examples/s]

✅ Dataset saved to 'resume_ner_dataset'


In [ ]:
from datasets import load_from_disk

# Load the dataset from the saved directory
dataset = load_from_disk("/content/resume_ner_dataset")

# Split the dataset
dataset = dataset.train_test_split(test_size=0.2)

print(dataset["train"][0])

{'input_ids': [101, 101, 12120, 108, 108, 1396, 1116, 108, 108, 177, 5329, 7756, 108, 108, 190, 5082, 117, 12247, 118, 18653, 108, 108, 170, 2723, 1143, 1113, 10364, 131, 5750, 119, 3254, 120, 187, 120, 12120, 108, 108, 1396, 1116, 108, 108, 177, 118, 5329, 120, 170, 108, 108, 128, 108, 108, 127, 108, 108, 173, 1181, 108, 108, 175, 108, 108, 127, 108, 108, 174, 108, 108, 1429, 108, 108, 121, 108, 108, 170, 108, 108, 128, 108, 108, 125, 108, 108, 171, 108, 108, 129, 3969, 108, 108, 2226, 1207, 7806, 117, 1702, 1106, 2866, 170, 8706, 2369, 1115, 1144, 1444, 1105, 2906, 6305, 1111, 19024, 119, 3969, 108, 108, 2226, 170, 1700, 1187, 146, 1169, 2867, 1114, 1139, 190, 108, 108, 189, 108, 108, 1211, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# To see the original text of a sample
print("Original text:", data[0]["content"])

# To see the tokens for the first sample in the training set
print("Tokens:", tokenizer.convert_ids_to_tokens(dataset["train"][0]["input_ids"]))

# To see the labels (NER tags) for the first sample in the training set
print("Labels:", [id2label[tag] for tag in dataset["train"][0]["labels"]])

Original text: Abhishek Jha
Application Development Associate - Accenture

Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a

• To work for an organization which provides me the opportunity to improve my skills
and knowledge for my individual and company's growth in best possible ways.

Willing to relocate to: Bangalore, Karnataka

WORK EXPERIENCE

Application Development Associate

Accenture -

November 2017 to Present

Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries
for the Bot which will be triggered based on given input. Also, Training the bot for different possible
utterances (Both positive and negative), which will be given as
input by the user.

EDUCATION

B.E in Information science and engineering

B.v.b college of engineering and technology -  Hubli, Karnataka

August 2013 to June 2017

12th in Mathematics

Woodbine modern school

April 2011 to March 2013

10th

Kendriya Vidyalaya

April 2001 to March 2011



# Task
Finetune a model for Named Entity Recognition (NER) using the provided dataset.

## Define training arguments

### Subtask:
Set up the training parameters such as the number of epochs, learning rate, batch size, and evaluation strategy.


**Reasoning**:
Import the `TrainingArguments` class and define the training parameters as a dictionary.



**Reasoning**:
The previous attempt to define `TrainingArguments` failed because `evaluation_strategy` is not a valid argument. I need to use the correct argument name, which is `eval_strategy`.



In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,           # Base LR
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=35,
    weight_decay=0.01,            # Regularization
    warmup_ratio=0.1,             # 10% of steps are warmup
    lr_scheduler_type="linear",   # Linear decay after warmup
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
)

#print(training_args)

## Initialize model for token classification

### Subtask:
Load a pre-trained model and configure it for token classification with the appropriate number of output labels based on the dataset.


**Reasoning**:
Initialize the model for token classification with the appropriate number of labels from the dataset.



In [ ]:
from transformers import AutoModelForTokenClassification
import torch
from collections import Counter
import numpy as np

# Calculate class weights
label_counts = Counter([label for sample in dataset["train"]["labels"] for label in sample])
total_labels = sum(label_counts.values())
# Calculate weights inversely proportional to class frequencies
# Add a small smoothing factor (e.g., 1) to avoid division by zero for rare labels
weights = [total_labels / (label_counts.get(i, 0) + 1) for i in range(len(id2label))]
# Normalize weights (optional but can help with learning rate tuning)
weights = weights / np.sum(weights)
class_weights = torch.tensor(weights, dtype=torch.float)


num_labels = len(id2label)
model = AutoModelForTokenClassification.from_pretrained(
    "yashpwr/resume-ner-bert-v2",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

# Assign class weights to the model's loss function
# Note: This assumes the model's loss function is CrossEntropyLoss or similar
# If using a custom loss function, you would need to modify that.
if hasattr(model, 'classifier') and hasattr(model.classifier, 'weight'):
    # Check if the model uses a standard classification head with a loss attribute
    if hasattr(model, 'base_model') and hasattr(model.base_model, 'embeddings'):
        # For models with a base_model attribute (like BERT)
        if hasattr(model, 'dropout') and hasattr(model, 'classifier'):
             # Assuming CrossEntropyLoss is used implicitly
             # We need to pass weights to the loss function during training,
             # which is typically handled by the Trainer when weights are part of the model.
             # A common way is to set the `weight` attribute of the loss function.
             # However, the Trainer directly uses the model's forward pass loss.
             # We can modify the model's forward pass or provide a custom Trainer.
             # A simpler approach with the standard Trainer is to pass weights to the model's config
             # or directly to the model if it supports a 'weight' parameter in its forward.
             # Let's try adding weights to the model's config, though Trainer support varies.
             # A more reliable way is often a custom Trainer or modifying the model's forward.

             # Let's try a common pattern: modifying the CrossEntropyLoss weight in the model's forward.
             # This requires modifying the model definition or its forward method.
             # Since we can't directly modify the class definition here,
             # a common workaround is to wrap the model or use hooks,
             # or rely on Trainer's potential support for weighted loss via model config.

             # Let's assume the model's forward pass accepts a 'weight' argument for loss calculation
             # or that the Trainer looks for a 'weight' attribute in the model.
             # This is not standard for AutoModelForTokenClassification with the default Trainer.
             # A more direct way is to modify the model's forward pass.
             # Given the limitations, let's stick to the most likely supported method
             # within the standard Trainer if it exists, or note the need for a custom approach.

             # The standard Trainer with AutoModelForTokenClassification does NOT directly support
             # passing class weights via the model's __init__ or config for the internal loss calculation.
             # To use weighted loss with the standard Trainer, you typically need to:
             # 1. Create a custom model class that inherits from AutoModelForTokenClassification
             #    and overrides the forward method to calculate weighted loss.
             # 2. Pass the class weights to this custom model's __init__.

             # Since we cannot define a new class or modify the existing one's forward directly here,
             # the standard Trainer approach for weighted loss in NER usually involves
             # ensuring the model's internal loss function (CrossEntropyLoss) is initialized with weights.
             # AutoModelForTokenClassification's default forward pass calculates loss using CrossEntropyLoss
             # but doesn't expose a way to pass weights easily via __init__ or config for this internal loss.

             # Let's revert to the model initialization without attempting to directly pass weights here,
             # as the standard Trainer does not support this out-of-the-box for the default loss.
             # We would need a custom Trainer or a custom model forward pass for weighted loss.
             pass # Revert the attempt to assign weights directly here.

# Acknowledging the need for a custom approach for weighted loss with the standard Trainer:
print("Note: Implementing weighted loss with the standard Hugging Face Trainer and AutoModelForTokenClassification typically requires a custom Trainer or modifying the model's forward pass.")
print("Class weights calculated but not applied to the standard model initialization.")


print(model.config.id2label)
print(model.config.label2id)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at yashpwr/resume-ner-bert-v2 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([25]) in the checkpoint and torch.Size([23]) in the model instantiated
- classifier.weight: found shape torch.Size([25, 768]) in the checkpoint and torch.Size([23, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Note: Implementing weighted loss with the standard Hugging Face Trainer and AutoModelForTokenClassification typically requires a custom Trainer or modifying the model's forward pass.
Class weights calculated but not applied to the standard model initialization.
{0: 'B-COLLEGE_NAME', 1: 'B-COMPANIES_WORKED_AT', 2: 'B-DEGREE', 3: 'B-DESIGNATION', 4: 'B-EMAIL_ADDRESS', 5: 'B-GRADUATION_YEAR', 6: 'B-LOCATION', 7: 'B-MOBILE_NUMBER', 8: 'B-NAME', 9: 'B-SKILLS', 10: 'B-YEARS_OF_EXPERIENCE', 11: 'I-COLLEGE_NAME', 12: 'I-COMPANIES_WORKED_AT', 13: 'I-DEGREE', 14: 'I-DESIGNATION', 15: 'I-EMAIL_ADDRESS', 16: 'I-GRADUATION_YEAR', 17: 'I-LOCATION', 18: 'I-MOBILE_NUMBER', 19: 'I-NAME', 20: 'I-SKILLS', 21: 'I-YEARS_OF_EXPERIENCE', 22: 'O'}
{'B-COLLEGE_NAME': 0, 'B-COMPANIES_WORKED_AT': 1, 'B-DEGREE': 2, 'B-DESIGNATION': 3, 'B-EMAIL_ADDRESS': 4, 'B-GRADUATION_YEAR': 5, 'B-LOCATION': 6, 'B-MOBILE_NUMBER': 7, 'B-NAME': 8, 'B-SKILLS': 9, 'B-YEARS_OF_EXPERIENCE': 10, 'I-COLLEGE_NAME': 11, 'I-COMPANIES_WORK

## Train the model

### Subtask:
Train the model using the Hugging Face `Trainer`.


In [ ]:
!pip install -q seqeval

In [ ]:
from transformers import Trainer, DataCollatorForTokenClassification
import numpy as np
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score

# Define the compute metrics function
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
        "accuracy": accuracy_score(true_labels, true_predictions),
    }

# Instantiate data collator
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

# Start training
trainer.train()

/tmp/ipython-input-2350822707.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: joshualayne246 (joshualayne246-myself) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


KeyboardInterrupt: 

## Evaluate the model

### Subtask:
Evaluate the trained model on the test set using appropriate metrics for NER, such as precision, recall, and F1-score.


**Reasoning**:
Evaluate the trained model on the test set using the `trainer.evaluate()` method and display the results.



In [ ]:
evaluation_results = trainer.evaluate()

print("Evaluation Results:")
print(evaluation_results)

## Inference

### Subtask:
Show how to use the fine-tuned model to perform Named Entity Recognition on new text.


**Reasoning**:
Import the pipeline function and create a NER pipeline with the trained model and tokenizer, then define sample text and use the pipeline to predict entities, and finally print the results.



In [ ]:
from transformers import pipeline

# Create a NER pipeline
# Use the trainer.model which was trained with augmented data
ner_pipeline = pipeline("token-classification", model=trainer.model, tokenizer=tokenizer, aggregation_strategy="simple")

# Define sample text
sample_text = """
John Smith
john.smith@gmail.com
Senior Software Engineer at Google
Skills: Python, JavaScript, Machine Learning
Education: B.Sc. Computer Science
"""

# Predict named entities
ner_results = ner_pipeline(sample_text)

# Print the results
print(ner_results)